bayesian inference example
A campaign on fb where clicks of users on the ad are success and not clicking is failure
The ad has been presented to 10 users so far, and 7 of the users have clicked on it. We would like to estimate the probability that the next user will click on the ad. 

In [ ]:
# import libraries
import numpy as np,pandas as pd
from scipy.misc import factorial
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,7)

In [ ]:
# functions
def BinomialDistProb(theta, n, x):
    """
    BinomialDistProb function for a binomial distribution
    n: [int] the number of experiments
    x: [int] the number of successes
    theta: [float] the proposed probability of success
    return prob. for given inputs
    """
    return (factorial(n) / (factorial(x) * factorial(n - x))) \
            * (theta ** x) * ((1 - theta) ** (n - x))

In [ ]:
# variables
#the number of impressions for our facebook-yellow-dress campaign
n_impressions = 10.
#the number of clicks for our facebook-yellow-dress campaign
n_clicks = 7.
#observed click through rate
ctr = n_clicks / n_impressions

In [ ]:
#0 to 1, all possible click through rates
possible_theta_valuesTemp =  list(range(0,100))
possible_theta_values = [x / 100 for x in possible_theta_valuesTemp]
print(possible_theta_values)

get likelihood function for binomial distribution; prob that 7 users out of 10 will open
facebook ad given the theta value

In [ ]:
likelihoods = []
for item in possible_theta_values:
    likelihoods.append(likelihood(item,10,7))
print(likelihoods)

In [ ]:
#pick the best theta, maximum likelihood estimator
mle = possible_theta_values[np.argmax(likelihoods)]
#plot
f, ax = plt.subplots(1)
ax.plot(possible_theta_values, likelihoods)
ax.axvline(mle, linestyle = "--")
ax.set_xlabel("Theta")
ax.set_ylabel("Likelihood")
ax.grid()
ax.set_title("Likelihood of Theta for New Campaign")
plt.show()

overlay this likelihood function with distribution of click-through rates from our previous 100 campaigns

In [ ]:
# randomly generate "true" clicks from beta distribution, 100 random samples(theta) from beta distribution 
theta_old = np.random.beta(11.5,48.5,size=100)
print(theta_old)

In [ ]:
# randomly generate corresponding number of user observations for each campaign
impressions_old = np.random.randint(1,10000,size=100)
print(impressions_old)

In [ ]:
# sample no of clicks for each campaign
clicks_old = np.random.binomial(impressions_old,theta_old).astype(float)
np.set_printoptions(precision=4)
print(clicks_old)

Previous campaigns vs current campaign

In [ ]:
#plot the histogram of previous click through rates with the evidence#of the new campaign
f, ax = plt.subplots(1)
ax.axvline(mle, linestyle = "--")
ax.plot(possible_theta_values, likelihoods)
zero_to_one = [j/100. for j in range(100)]
counts, bins = np.histogram((clicks_old/impressions_old)
                            , bins=zero_to_one)
counts = counts / 100.
ax.plot(bins[:-1],counts, alpha = .5)
line1, line2, line3 = ax.lines
ax.legend((line2, line3), ('Likelihood of Theta for New Campaign'
                           , 'Frequency of Theta Historically')
                          , loc = 'upper left')
ax.set_xlabel("Theta")
ax.grid()
ax.set_title("Evidence vs Historical Click Through Rates")
plt.show()

Choosing beta distribution as a prior for theta, we will fit the distribution to the old campaign data to get the distribution parameters (alpha,beta)

In [ ]:
from scipy.stats import beta
#fit beta to previous CTRs
prior_parameters = beta.fit(theta_old, floc = 0, fscale = 1)

In [ ]:

#extract alpha,beta from fit
alpha, betavar = prior_parameters[0:2]

#define prior distribution sample from prior
prior_distribution = beta(alpha, betavar)
#get histogram of samples
prior_samples = prior_distribution.rvs(10000)
#get histogram of samples
fit_counts, bins = np.histogram(prior_samples
                                , zero_to_one)
print(fit_counts)
print(zero_to_one)

In [ ]:
# normalize histogram
normalized_fit_counts = []
for item in fit_counts:
    normalized_fit_counts.append(float(item)/sum(fit_counts))  
print(normalized_fit_counts)

In [ ]:
#plot
f, ax = plt.subplots(1)
ax.plot(bins[:-1], normalized_fit_counts)
hist_ctr, bins = np.histogram(theta_old
                              , zero_to_one)
hist_ctr = hist_ctr[:]/np.sum(hist_ctr)
ax.plot(bins[:-1], hist_ctr)
estimated_prior, previous_click_through_rates = ax.lines
ax.legend((estimated_prior, previous_click_through_rates)
          ,('Estimated Prior'
            , 'Previous Click Through Rates'))
ax.grid()
ax.set_title("Comparing Empirical Prior with Previous Click Through Rates")
plt.show()

In [ ]:
import pymc3 as pm

#create our data:clicks = np.array([n_clicks])
#clicks represents our successes. We observed 7 clicks.impressions = np.array([n_impressions])
#this represents the number of trials. There were 10 impressions.

with pm.Model() as model:
#sets a context; all code in block "belongs" to the model object

    theta_prior = pm.Beta('prior', 11.5, 48.5)
    #our prior distribution, Beta (11.5, 48.5)
    observations = pm.Binomial('obs',n = impressions
                               , p = theta_prior
                               , observed = clicks)     #Sampling distribition of outcomes in the dataset.
    #our prior p_prior will be updated with data


    start = pm.find_MAP()    #find good starting values for the sampling algorithm
    #Max Aposterior values, or values that are most likely

    step = pm.NUTS(state=start)     #Choose a particular MCMC algorithm     
    #we'll choose NUTS, the No U-Turn Sampler (Hamiltonian)

    trace = pm.sample(5000
                      , step
                      , start=start
                      , progressbar=True)               #obtain samples

In [ ]:
#plot the histogram of click through rates
plt.rcParams['figure.figsize'] = (16, 7)
#get histogram of samples from posterior distribution of CTRs
posterior_counts, posterior_bins = np.histogram(trace['prior']
                                                ,bins=zero_to_one)
#normalized histogramposterior_counts = posterior_counts / float(posterior_counts.sum())
#take the mean of the samples as most plausible value
most_plausible_theta = np.mean(trace['prior'])
#histogram of samples from prior distribution
prior_counts, bins = np.histogram(prior_samples
                                  , zero_to_one)#normalize
prior_counts = map(lambda x: float(x)/prior_counts.sum()
                   , prior_counts)
#plot
f, ax = plt.subplots(1)
ax.plot(possible_theta_values, likelihoods)
ax.plot(bins[:-1],prior_counts, alpha = .2)
ax.plot(bins[:-1],posterior_counts)
ax.axvline(most_plausible_theta, linestyle = "--", alpha = .2)
line1, line2, line3, line4 = ax.lines
ax.legend((line1, line2, line3, line4), ('Evidence'
                                         , 'Prior Probability for Theta'
                                         , 'Posterior Probability for Theta'
                                         , 'Most Plausible Theta'
                                        ), loc = 'upper left')
ax.set_xlabel("Theta")
ax.grid()
ax.set_title("Prior Distribution Updated with Some Evidence")
plt.show()

In [ ]:
f, ax = plt.subplots(1)
ax.plot(bins[:-1],prior_counts, alpha = .2)

counts = {}
for ad_impressions in [10, 100, 1000, 10000]:
    trace = traces[ad_impressions]
    posterior_counts, posterior_bins = np.histogram(trace['prior'], bins=[j/100. for j in xrange(100)])
    posterior_counts = posterior_counts / float(len(trace))
    ax.plot(bins[:-1], posterior_counts)
line0, line1, line2, line3, line4 = ax.lines
ax.legend((line0, line1, line2, line3, line4), ('Prior Distribution'
                                                ,'Posterior after 10 Impressions'
                                                , 'Posterior after 100 Impressions'
                                                , 'Posterior after 1000 Impressions'
                                                ,'Posterior after 10000 Impressions'))
ax.set_xlabel("Theta")
ax.axvline(ctr, linestyle = "--", alpha = .5)
ax.grid()
ax.set_ylabel("Probability of Theta")
ax.set_title("Posterior Shifts as Weight of Evidence Increases")
plt.show()